In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/ng-video-lecture/venv/lib/python3.11/site-packages')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
from typing import List
import math
import numpy as np
import matplotlib.pyplot as plt

In [3]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# Thinking about how layernorm affects these hierarchical vectors

In [ ]:
### gotta turn this into a way to keep each sub-vector on the unit hypersphere

import numpy as np
from scipy.stats import shapiro

def adjust_and_concatenate(v1, v2):
    """
    Concatenates two vectors and adjusts the second vector so that the entire concatenated vector
    is normally distributed with a mean of 0 and standard deviation of 1, without altering the first vector.
    
    Parameters:
    - v1: First vector of length 100, already normalized.
    - v2: Second vector of length 100, already normalized.
    
    Returns:
    - concatenated_vector: Concatenated vector of length 200, with adjusted values for the second half.
    """
    # Step 1: Concatenate the vectors
    concatenated_vector = np.concatenate([v1, v2])
    
    # Step 2: Adjust the second half to maintain overall normal distribution
    # Calculate mean and std of the concatenated vector
    mean_full = np.mean(concatenated_vector)
    std_full = np.std(concatenated_vector)
    
    # Calculate adjustments needed for the second half
    adjustment_factor = 1 / std_full  # To ensure overall std is 1
    mean_adjustment = -mean_full * adjustment_factor  # To ensure overall mean is 0
    
    # Apply adjustments to the second half
    concatenated_vector[100:] = concatenated_vector[100:] * adjustment_factor + mean_adjustment
    
    return concatenated_vector


def test_concatenation(concatenated_vector, original_vec1):
    """
    Tests whether the concatenated vector maintains the first 100 values unchanged
    and whether the entire 200-element vector follows a normal distribution.

    Parameters:
    - concatenated_vector: The concatenated and adjusted vector of length 200.
    - original_vec1: The original first vector to compare the first 100 values against.

    Returns:
    - first_half_unchanged: Boolean, True if the first 100 values are unchanged.
    - is_normal: Boolean, True if the concatenated vector is normally distributed.
    """
    # Check if the first 100 values are unchanged
    first_half_unchanged = np.array_equal(concatenated_vector[:100], original_vec1)

    # Test for normality on the entire vector
    stat, p = shapiro(concatenated_vector)
    is_normal = p > 0.05  # Using a significance level of 0.05

    return first_half_unchanged, is_normal

# Example usage with dummy data
vec1 = np.random.normal(0, 1, 100)  # First vector, normalized
vec2 = np.random.normal(0, 1, 100)  # Second vector, normalized

# Concatenate and adjust
concatenated_vector = adjust_and_concatenate(vec1, vec2)

# Test the concatenation
first_half_unchanged, is_normal = test_concatenation(concatenated_vector, vec1)

first_half_unchanged, is_normal

In [ ]:
import numpy as np
from scipy.stats import shapiro, norm
import matplotlib.pyplot as plt

# Function to generate random vectors using different distributions
def generate_random_vectors(length, n_vectors):
    vectors = {
        "normal": np.random.normal(size=(n_vectors, length)),
        "uniform": np.random.uniform(size=(n_vectors, length)),
        "exponential": np.random.exponential(scale=1.0, size=(n_vectors, length)),
        # Add more distributions if needed
    }
    return vectors

# Function to apply layer normalization to vectors
def layer_normalize(vectors):
    normed_vectors = {}
    for key, vecs in vectors.items():
        mean = np.mean(vecs, axis=1, keepdims=True)
        std = np.std(vecs, axis=1, keepdims=True)
        normed_vectors[key] = (vecs - mean) / std
    return normed_vectors

# Function to create subsets and test for normality
def test_subsets_for_normality(vectors, max_length):
    results = {}
    subset_lengths = [2**i for i in range(3, int(np.log2(max_length))+1)]  # 8, 16, 32, ..., up to max_length
    for key, vecs in vectors.items():
        results[key] = {}
        for length in subset_lengths:
            p_values = []
            for vec in vecs:
                subset = vec[:length]
                _, p_value = shapiro(subset)
                p_values.append(p_value)
            results[key][length] = np.mean(p_values)
    return results

# Parameters
length = 1024
n_vectors = 1000  # Number of vectors for each distribution

# Generate random vectors
random_vectors = generate_random_vectors(length, n_vectors)

# Apply layer normalization
normalized_vectors = layer_normalize(random_vectors)

# Test subsets for normality
results = test_subsets_for_normality(normalized_vectors, length)

results


In [ ]:
# Function to plot histograms for original and normalized vectors
def plot_histograms(original_vectors, normalized_vectors, distribution):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
    fig.suptitle(f'{distribution.capitalize()} Distribution: Before and After Normalization')

    # Original Vector Histogram
    axes[0].hist(original_vectors[distribution][0], bins=30, alpha=0.7, color='blue')
    axes[0].set_title('Original')
    axes[0].set_xlabel('Value')
    axes[0].set_ylabel('Frequency')

    # Normalized Vector Histogram
    axes[1].hist(normalized_vectors[distribution][0], bins=30, alpha=0.7, color='green')
    axes[1].set_title('Normalized')
    axes[1].set_xlabel('Value')

    plt.show()

# Plot for each distribution
for distribution in ["normal", "uniform", "exponential"]:
    plot_histograms(random_vectors, normalized_vectors, distribution)

In [ ]:
# Function to calculate Euclidean norms for vectors
def calculate_norms(vectors):
    norms = {}
    for key, vecs in vectors.items():
        norms[key] = np.linalg.norm(vecs, axis=1)  # Euclidean norm (L2 norm) of each vector
    return norms

# Calculate norms for original and normalized vectors
original_norms = calculate_norms(random_vectors)
normalized_norms = calculate_norms(normalized_vectors)

# Plotting the magnitudes using box plots
def plot_magnitude_comparison(original_norms, normalized_norms):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    distributions = ['normal', 'uniform', 'exponential']

    for i, distribution in enumerate(distributions):
        # Combine original and normalized norms for plotting
        data = [original_norms[distribution], normalized_norms[distribution]]
        axes[i].boxplot(data, patch_artist=True, labels=['Original', 'Normalized'])
        axes[i].set_title(f'{distribution.capitalize()} Distribution')
        axes[i].set_ylabel('Magnitude (Euclidean Norm)')

    plt.tight_layout()
    plt.show()

plot_magnitude_comparison(original_norms, normalized_norms)